In [218]:
import pickle
import dgl
import torch
from torch.utils.data import DataLoader

In [219]:
with open('./train_data.pkl', 'rb') as f:
    data = pickle.load(f)
with open('./test_data.pkl', 'rb') as f1:
    data1 = pickle.load(f1)
traindata = []
testdata = []
for i in data:
    tmp = i.ndata['y'][0]
    traindata.append((i, tmp))
for j in data1:
    tmp = j.ndata['y'][0]
    testdata.append((j, tmp))

def collate(samples):
    # 输入`samples` 是一个列表
    # 每个元素都是一个二元组 (图, 标签)
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.Tensor(labels).type(torch.long)

data_loader = DataLoader(traindata, batch_size=5, shuffle=True,
                         collate_fn=collate)
test_loader = DataLoader(testdata, batch_size=5, shuffle=True,
                         collate_fn=collate)

In [220]:
import dgl.function as fn
import torch
import torch.nn as nn

# 将节点表示h作为信息发出
msg = fn.copy_src(src='feat', out='m')

def reduce(nodes):
    """对所有邻节点节点特征求平均并覆盖原本的节点特征。"""
    accum = torch.mean(nodes.mailbox['m'], 1)
    return {'feat': accum}

class NodeApplyModule(nn.Module):
    """将节点特征 hv 更新为 ReLU(Whv+b)."""
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation

    def forward(self, node):
        h = self.linear(node.data['feat'])
        h = self.activation(h)
        return {'feat' : h}

In [221]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCNLayer, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        # 使用 h 初始化节点特征。
        g.ndata['feat'] = feature
        # 使用 update_all接口和自定义的消息传递及累和函数更新节点表示。
        g.update_all(msg, reduce)
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('feat')

In [225]:
import torch.nn.functional as F

class Regress(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(Regress, self).__init__()
        # 两层图卷积层。
        self.layers = nn.ModuleList([
            GCNLayer(in_dim, hidden_dim, F.relu),
            GCNLayer(hidden_dim, hidden_dim, F.relu),
            GCNLayer(hidden_dim, hidden_dim, F.relu),
            GCNLayer(hidden_dim, hidden_dim, F.relu)])
        # 回归预测层。
        self.emb = nn.Linear(hidden_dim, hidden_dim)
        self.emb1 = nn.Linear(hidden_dim, hidden_dim)
        self.regress = nn.Linear(hidden_dim, 5)

    def forward(self, g):
        # 初始数据
        h = g.ndata['feat']
        # 图卷积层。
        for conv in self.layers:
            h = conv(g, h)
        g.ndata['feat'] = h
        # 读出函数。
        graph_repr = dgl.mean_nodes(g, 'feat')
        # 回归预测层。
        graph_repr = self.emb(graph_repr)
        graph_repr = self.emb1(graph_repr)
        return F.softmax(self.regress(graph_repr))

In [242]:
import torch.optim as optim
# 创建模型
model = Regress(13, 128)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()

epoch_losses = []
tmp_loss = 1.61
for epoch in range(20):
    epoch_loss = 0
    for iter, (bg, label) in enumerate(data_loader):
        prediction = model(bg)
        loss = loss_func(prediction, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.detach().item()
    epoch_loss /= (iter + 1)
    if epoch_loss <= tmp_loss:
        tmp_loss = epoch_loss
        model_res = model
    print('Epoch {}, loss {:.4f}'.format(epoch, epoch_loss))
    epoch_losses.append(epoch_loss)

<ipython-input-225-10dd0d367261>:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.regress(graph_repr))


Epoch 0, loss 1.6115
Epoch 1, loss 1.6107
Epoch 2, loss 1.6098
Epoch 3, loss 1.6100
Epoch 4, loss 1.6103
Epoch 5, loss 1.6099
Epoch 6, loss 1.6100
Epoch 7, loss 1.6093
Epoch 8, loss 1.6191
Epoch 9, loss 1.6475
Epoch 10, loss 1.6127
Epoch 11, loss 1.6096
Epoch 12, loss 1.6085
Epoch 13, loss 1.6034
Epoch 14, loss 1.5876
Epoch 15, loss 1.5551
Epoch 16, loss 1.5818
Epoch 17, loss 1.5554
Epoch 18, loss 1.5380
Epoch 19, loss 1.5395


In [243]:
with torch.no_grad():
    model_res.eval()
    epoch_loss = 0
    erer = 0
    for iter, (bg, label) in enumerate(test_loader):
        prediction = model_res(bg)
        oooo = torch.sum(prediction.argmax(dim=1) == label)
        erer += oooo
        loss = loss_func(prediction, label)
        epoch_loss += loss.detach().item()
    epoch_loss /= (iter + 1)
    print('loss {:.4f}'.format(epoch_loss))
    print(erer)

loss 1.4885
tensor(12)


<ipython-input-225-10dd0d367261>:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.regress(graph_repr))
